In [ ]:
import wandb
import os
import json
import pandas as pd

# Connect to your W&B workspace
api = wandb.Api()

# Define your project
entity = "hoanglongdang2001"  # e.g. "myusername" or your org
project = "hrm-rl-dqn-maze-full-observability"  # e.g. "my-cool-experiment"

# Optional: filter runs based on config
filters = {
    "config.arch_exclude_data_dependent.H_layers": 4,
    "config.dataset.buffer_capacity": 1048576,
    "config.use_target_network": True,
    "config.eps_decay_steps": 300000,
    "config.end_eps": 0.33,
}

# Fetch runs
runs = api.runs(f"{entity}/{project}", filters=filters)

# Create export directory
export_dir = "outputs/wandb_export"
os.makedirs(export_dir, exist_ok=True)

# iterate and export
for run in runs:
    run_dir = os.path.join(export_dir, run.name or run.id)
    os.makedirs(run_dir, exist_ok=True)

    # ---- Config ----
    with open(os.path.join(run_dir, "config.json"), "w") as f:
        json.dump(run.config, f, indent=2)

    # ---- Summary ----
    with open(os.path.join(run_dir, "summary.json"), "w") as f:
        json.dump(run.summary._json_dict, f, indent=2)

    # ---- Metadata ----
    with open(os.path.join(run_dir, "metadata.json"), "w") as f:
        json.dump(run.metadata, f, indent=2)

    # ---- History (metrics over time) ----
    validation_history = pd.DataFrame(
        run.history(
            samples=10000,
            keys=[
                "val_avg_reward",
                "val_avg_episode_length",
                "val_frac_envs_terminated_reward_1",
            ],
        )
    )
    validation_history.to_csv(
        os.path.join(run_dir, "validation_events.csv"), index=False
    )

    training_history = pd.DataFrame(
        run.history(
            samples=10000,
            keys=[
                "train_batch_avg_reward",
                "train_loss",
                "exploration_eps",
                "model_lr",
            ],
        )
    )
    training_history.to_csv(os.path.join(run_dir, "training_events.csv"), index=False)

    print(f"Exported {run.name or run.id}")

print(f"\nAll exports saved to: {os.path.abspath(export_dir)}")